In [ ]:
levels = 5

lulcmap = 'cosc2021'

iclasses = ['620']

expclasses = {
    'pure_positive' : 'water',
    'pure_negative' : 'no_water',
    'mixed' : 'mix_water'
    #'mixed_75_100'  : 'watermix_75_100',
    #'mixed_50_75'   : 'watermix_50_75',
    #'mixed_25_50'   : 'watermix_25_50',
    #'mixed_0_25'    : 'watermix_0_25'
}

db = 'gthruth_valeflores_v3'

out_tbl = '/mnt/d/autocls_e3/trainref/pixels_exp32.xlsx'

In [ ]:
import numpy as np

from glass.sql.q import q_to_obj
from glass.dtt.mge.pd import merge_df
from glass.wt import obj_to_tbl

In [ ]:
icls_str = ', '.join([f"'{c}'" for c in iclasses])
icls_array = f"ARRAY[{icls_str}]"

classes = q_to_obj(db, (
    "SELECT mcls.* "
    "FROM mapclasses AS mcls "
    "INNER JOIN refmaps AS rmap "
    "ON mcls.mapfk = rmap.mapid "
    f"WHERE rmap.slug = '{lulcmap}' "
    f"AND mcls.code <> ANY({icls_array})"
))

nicls_str = ', '.join([f"'{c}'" for c in classes.code.tolist()])
nicls_array = f"ARRAY[{nicls_str}]"

In [ ]:
lquery = []

for i in range(1, levels):

    if i == 1:
        q = (
            f"SELECT cid AS root{str(i+1)}, rootid "
            "FROM gthruth_fishnet AS tbl "
            f"WHERE tbl.level = {str(i+1)}"
        )

    else:
        q = (
            f"SELECT tbl.cid AS root{str(i+1)}, jtb.rootid, "
            "ST_Area(tbl.geom) AS geom "
            "FROM gthruth_fishnet AS tbl "
            f"INNER JOIN ({lquery[i-2]}) AS jtb "
            f"ON tbl.rootid = jtb.root{str(i)} "
            f"WHERE tbl.level = {str(i+1)}"
        )

    lquery.append(q)

In [ ]:
flq = []

cls_view = (
    "SELECT * "
    "FROM pxclasses AS pxcls "
    "INNER JOIN ("
        "SELECT mcls.*, rmap.slug "
        "FROM mapclasses AS mcls "
        "INNER JOIN refmaps AS rmap "
        "ON mcls.mapfk = rmap.mapid "
        f"WHERE rmap.slug = '{lulcmap}'"
    ") AS tcls "
    "ON pxcls.clsid = tcls.classid"
)


In [ ]:
for i in range(levels):
    if i == 0 or i == 1:
        rootid = "px.cid AS rootid" if not i else "px.rootid"
        lvl = "1" if not i else "2"

        q = (
            f"SELECT {rootid}, ST_Area(geom) AS geom, pxcls.classid, "
            "pxcls.slug, pxcls.code, pxcls.classname "
            "FROM gthruth_fishnet AS px "
            f"INNER JOIN ({cls_view}) AS pxcls "
            "ON px.cid = pxcls.pxid "
            f"WHERE px.level = {lvl}"
        )
    else:
        q = (
            "SELECT px.rootid, px.geom, pxcls.classid, "
            "pxcls.slug, pxcls.code, pxcls.classname "
            f"FROM ({lquery[i-1]}) AS px "
            f"INNER JOIN ({cls_view}) AS pxcls "
            f"ON px.root{str(i+1)} = pxcls.pxid"
        )

    flq.append(q)

In [ ]:
_fq = " UNION ALL ".join(flq)

fq = f"SELECT * FROM ({_fq}) AS allview ORDER BY allview.rootid"

In [ ]:
# Get Pure Positive Pixels

ppq = (
    f"SELECT rootid, '{expclasses['pure_positive']}' AS expclass, "
    "ARRAY_AGG(json_build_object("
    	"'classid', classid, "
    	"'classcode', code, "
    	"'classname', classname, "
    	"'geom', geom"
    ")) AS classes, "
    "(ARRAY_AGG(classmax) FILTER (WHERE classmax IS NOT NULL))[1] AS classmax "
    "FROM ("
        "SELECT *, "
        "SUM(geom) OVER(PARTITION BY rootid ORDER BY rootid) AS geomsum, "
        "CASE "
            "WHEN geom = MAX(geom) OVER(PARTITION BY rootid ORDER BY rootid) "
            "THEN code ELSE NULL "
        "END AS classmax "
        "FROM ("
            "SELECT rootid, code, classid, classname, "
            "SUM(geom) AS geom "
            f"FROM ({_fq}) AS lulc "
            f"WHERE code = ANY({icls_array}) "
            "GROUP BY rootid, code, classid, classname"
        ") AS foo"
    ") AS foo2 "
    "WHERE geomsum = 100 "
    "GROUP BY rootid "
    "ORDER BY rootid"
)

pnq = (
    f"SELECT rootid, '{expclasses['pure_negative']}' AS expclass, "
    "ARRAY_AGG(json_build_object("
    	"'classid', classid, "
    	"'classcode', code, "
    	"'classname', classname, "
    	"'geom', geom"
    ")) AS classes, "
    "(ARRAY_AGG(classmax) FILTER (WHERE classmax IS NOT NULL))[1] AS classmax "
    "FROM ("
        "SELECT *, "
        "SUM(geom) OVER(PARTITION BY rootid ORDER BY rootid) AS geomsum, "
        "CASE "
            "WHEN geom = MAX(geom) OVER(PARTITION BY rootid ORDER BY rootid) "
            "THEN code ELSE NULL "
        "END AS classmax "
        "FROM ("
            "SELECT rootid, code, classid, classname, "
            "SUM(geom) AS geom "
            f"FROM ({_fq}) AS lulc "
            f"WHERE code <> ANY({icls_array}) "
            "GROUP BY rootid, code, classid, classname"
        ") AS foo"
    ") AS foo2 "
    "WHERE geomsum = 100 "
    "GROUP BY rootid "
    "ORDER BY rootid"
)

mixcase = (
    f"'{expclasses['mixed']}' AS expclass"
) if 'mixed' in expclasses else (
    "CASE "
    	"WHEN igeom >= 75 "
    	f"THEN '{expclasses['mixed_75_100']}' ELSE CASE "
    		f"WHEN igeom < 75 AND igeom >= 50 "
    		f"THEN '{expclasses['mixed_50_75']}' ELSE CASE "
    			f"WHEN igeom < 50 AND igeom >= 25 "
    			f"THEN '{expclasses['mixed_25_50']}' ELSE '{expclasses['mixed_0_25']}' "
    		"END "
    	"END "
    "END AS expclass"
)

mixq = (
    f"SELECT {mixcase}, rootid, classes, igeom, classmax "
    "FROM ("
    	"SELECT rootid, ARRAY_AGG(code) AS code, "
    	"SUM(igeom) AS igeom, "
    	"ARRAY_AGG(json_build_object("
    		"'classid', classid, "
    		"'classcode', code, "
    		"'classname', classname, "
    		"'geom', geom "
    	")) AS classes, "
        "(ARRAY_AGG(classmax) FILTER (WHERE classmax IS NOT NULL))[1] AS classmax "
    	"FROM ("
    		"SELECT rootid, code, classid, classname, "
    		"SUM(geom) AS geom, "
    		"SUM(igeom) AS igeom, "
            "CASE "
                "WHEN SUM(geom) = MAX(SUM(geom)) OVER(PARTITION BY rootid ORDER BY rootid) "
                "THEN code ELSE NULL "
            "END AS classmax "
    		"FROM ("
    			"SELECT *, CASE "
    				f"WHEN code = ANY({icls_array}) "
    				"THEN geom ELSE 0 "
    			"END AS igeom "
    			f"FROM ({_fq}) AS finalq"
    		") AS lulc "
    		"GROUP BY rootid, code, classid, classname"
    	") AS foo "
    	"GROUP BY rootid"
    ") AS foo "
    f"WHERE code && {icls_array} "
    f"AND code && {nicls_array}"
)

In [ ]:
pp_df = q_to_obj(db, ppq)

pn_df = q_to_obj(db, pnq)

mix_df = q_to_obj(db, mixq)

fdf = merge_df([pp_df, pn_df, mix_df], out_tbl)

fdf['igeom'] = np.where(
    fdf['igeom'].isna(),
    'NoData', fdf['igeom']
)

obj_to_tbl(fdf, out_tbl, sheetsName=lulcmap)